# Assignment 5 - Scott Berry

## Importing the libraries


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset
The y values are the binary yes/no application acceptance

The shape of the dataset is set to the num variables

In [2]:
dataset = pd.read_csv('Credit_Card_Applications.csv')
X = dataset.iloc[:, :-1].values 
y = dataset.iloc[:, -1].values

num_applications = dataset.shape[0]
num_categories = dataset.shape[1]

## Feature Scaling
This normalizes the data to put all values between 0 and 1

In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
X = sc.fit_transform(X)

## Split into train/test and convert to Torch tensors
Data is split 90/10 into Torch tensors


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
training_set = np.hstack((X_train, y_train.reshape((y_train.shape[0],1))))
test_set = np.hstack((X_test, y_test.reshape((y_test.shape[0],1))))
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Create AutoEncoder Neural Network
This model inherits from Torch Neural Network with some modified values/methods

Different optimizers affect model loss due to their effect on learning rate and weights

The example on Canvas used the RMSprop optimizer (a gradient descent variant) due to ability to increase learning rate reliably and as such will be used in this notebook

The Adam optimizer is the ideal choice in most datasets, however, the main reasons being the faster compute time which can increase the number of epochs and easier parameter tuning

In [5]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(num_categories, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, num_categories)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Train the AutoEncoder
AE model is trained over 200 epochs for each value in the train set

With too few epochs loss is not minimized, too many and the model will be over-fitted

In [6]:
num_epoch = 200
for epoch in range(1, num_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(num_applications):
        try:
            input = Variable(training_set[id_user]).unsqueeze(0)
            target = input.clone()
            if torch.sum(target.data > 0) > 0:
                output = sae(input)
                target.require_grad = False
                output[target == 0] = 0
                loss = criterion(output, target)
                mean_corrector = num_categories / float(torch.sum(target.data > 0) + 1e-10)
                loss.backward()
                train_loss += np.sqrt(loss.data*mean_corrector)
                s += 1.
                optimizer.step()
        except IndexError:
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+ str(train_loss/s))

epoch: 1 loss: tensor(0.1542)
epoch: 2 loss: tensor(0.1410)
epoch: 3 loss: tensor(0.1398)
epoch: 4 loss: tensor(0.1397)
epoch: 5 loss: tensor(0.1397)
epoch: 6 loss: tensor(0.1396)
epoch: 7 loss: tensor(0.1396)
epoch: 8 loss: tensor(0.1395)
epoch: 9 loss: tensor(0.1393)
epoch: 10 loss: tensor(0.1391)
epoch: 11 loss: tensor(0.1387)
epoch: 12 loss: tensor(0.1381)
epoch: 13 loss: tensor(0.1372)
epoch: 14 loss: tensor(0.1359)
epoch: 15 loss: tensor(0.1343)
epoch: 16 loss: tensor(0.1327)
epoch: 17 loss: tensor(0.1315)
epoch: 18 loss: tensor(0.1301)
epoch: 19 loss: tensor(0.1290)
epoch: 20 loss: tensor(0.1276)
epoch: 21 loss: tensor(0.1261)
epoch: 22 loss: tensor(0.1248)
epoch: 23 loss: tensor(0.1237)
epoch: 24 loss: tensor(0.1229)
epoch: 25 loss: tensor(0.1225)
epoch: 26 loss: tensor(0.1223)
epoch: 27 loss: tensor(0.1220)
epoch: 28 loss: tensor(0.1218)
epoch: 29 loss: tensor(0.1216)
epoch: 30 loss: tensor(0.1215)
epoch: 31 loss: tensor(0.1214)
epoch: 32 loss: tensor(0.1213)
epoch: 33 loss: t

## Test the AutoEncoder
The testing computes a total test loss value by comparing the test set to output of the AE model

The relatively low test loss result indicates that this classifier can reliably predict which applications will be approved/disapproved

The frauds in the test set are outputted based on approvals that SHOULD have been failures set at a threshold of 0.08 loss

In [7]:
test_loss = 0
s = 0.
frauds = []
for id_user in range(num_applications):
    try:
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = Variable(test_set[id_user]).unsqueeze(0)
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            if loss > 0.08:
                frauds.append(id_user)
            mean_corrector = num_categories / float(torch.sum(target.data > 0) + 1e-10)
            test_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
    except IndexError:
        s += 1.
print('test loss: '+str(test_loss/s))
print("frauds: " + str(frauds))

test loss: tensor(0.0177)
frauds: [3]


## Create the Boltzmann Machine
This RBM class is created by specifying weights, hidden and visible nodes

Further, the training method is specified here with batches of 100

In [8]:
class RBM:
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

## Train the Boltzmann Machine
The RBM model is trained over the batches of the training set for the length of the training set

In [9]:
num_epoch = 7
for epoch in range(1, num_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, num_applications - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.4102)
epoch: 2 loss: tensor(0.3710)
epoch: 3 loss: tensor(0.3737)
epoch: 4 loss: tensor(0.3675)
epoch: 5 loss: tensor(0.3679)
epoch: 6 loss: tensor(0.3768)
epoch: 7 loss: tensor(0.3771)


## Test the Boltzmann Machine
The testing set is compared to the RBM model

The high test loss result is indicative of the model not performing as well as the AE model

Each fraud is printed when loss is found at a threshold of 0.50

In [10]:
test_loss = 0
s = 0.
frauds = []
for id_user in range(num_applications):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        if torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) > 0.50:
            frauds.append(id_user)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))
print("frauds: " + str(frauds))

test loss: tensor(0.3764)
frauds: [2, 39, 48, 63, 65]
